In [1]:
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv

In [2]:
!pip install --quiet huggingface_hub pandas datasets

In [3]:
import pandas as pd

# load CSV
train_df = pd.read_csv("train.csv", header=None, names=["label","title","description"])
test_df  = pd.read_csv("test.csv",  header=None, names=["label","title","description"])

# merge title+description
train_df["text"] = train_df["title"].str.strip() + ". " + train_df["description"].str.strip()
test_df["text"]  = test_df["title"].str.strip()  + ". " + test_df["description"].str.strip()

In [4]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Tokenize a small batch
batch = list(train_df["text"].iloc[:8])
encoding = tokenizer(batch,
                     padding="max_length",
                     truncation=True,
                     max_length=128,
                     return_tensors="pt")

print("input_ids:", encoding["input_ids"].shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


input_ids: torch.Size([8, 128])


In [5]:
import torch

torch.save(encoding, "tokenized_sample.pt")


In [8]:
# # 1) (Re)install & upgrade to ensure a clean cache
# !pip install --quiet --upgrade datasets transformers

# # 2) Load AG News fresh into a Colab-local cache
# from datasets import load_dataset

# dataset = load_dataset(
#     "ag_news",
#     cache_dir="/tmp/hf_datasets",       # Colab’s ephemeral cache
#     download_mode="force_redownload"    # ignore any old cache
# )
# train_ds = dataset["train"]
# test_ds  = dataset["test"]

# # 3) Apply your tokenizer function over the full datasets
# tokenized_train = train_ds.map(tokenize_batch, batched=True)
# tokenized_test  = test_ds.map(tokenize_batch, batched=True)

# # 4) Set the PyTorch format so Trainer can consume it directly
# tokenized_train.set_format("torch",
#     columns=["input_ids", "attention_mask", "label"])
# tokenized_test.set_format("torch",
#     columns=["input_ids", "attention_mask", "label"])

# # 5) Quick sanity check on a tokenized example
# print(tokenized_train[0])